In [43]:
# 0. 패키지 설치 (최초 1회)
# LangChain 핵심
!pip install langchain langchain-core langchain-community langchain-text-splitters langchain-experimental

# Google Gemini + LangSmith 추적
!pip install langchain-google-genai langsmith

# 벡터 DB + 임베딩
!pip install chromadb sentence-transformers tf-keras

# 지식그래프 구조 관리
!pip install networkx

In [1]:
# 1. 환경 설정
import os
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI

# API Key 설정
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("GOOGLE_API_KEY: ")
if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LANGSMITH_API_KEY: ")

# LangSmith 추적 활성화
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "petsurance-rag"

# 모델 초기화
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
print("환경 설정 완료")

환경 설정 완료


In [2]:
# 2. 문서 로드
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# DirectoryLoader: 디렉터리의 파일을 Document 객체로 변환
# glob 패턴으로 특정 확장자 파일을 일괄 로드 가능
loader = DirectoryLoader(
    path="data/petsurance_markdown",
    glob="**/*.md",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)

documents = loader.load()
print(f"{len(documents)}개 문서 로드 완료")

# 샘플 확인
print(f"\n[출처]: {documents[0].metadata['source']}")
print(f"[내용]:\n{documents[0].page_content[:500]}...")


122개 문서 로드 완료

[출처]: data\petsurance_markdown\Carrot\Carrot_SmartON_PetWalkIns_Method.md
[내용]:
사업방법서  별지

캐롯  스마트ON  펫산책보험

1. 보험의 종류 : 특종/기타/동물

2. 보험종목의 명칭 : 캐롯 스마트ON 펫산책보험

3. 보험의 목적

   가. 법률상의 배상책임

      나.  각종 비용손해

4. 보험기간 및 보험료 납입주기에 관한 사항

      가.  보험기간  :  1년

   나. 보험료 납입주기 : 일시납을 원칙으로 하되, 해외여행 스마트ON 특별약관을

부가하는 계약의 경우 정산보험료를 영수할 수 있다.

5. 배당에 관한 사항 : 배당금을 지급하지 않는다.

6. 보험료 차등적용에 관한 사항

   가. ｢보험료 및 해약환급금 산출방법서｣에서 정한 바에 따라 적용한다.

7. 의무가입에 관한 사항 (1)

   가. 의무가입대상 : 펫산책 스마트ON 특별약관

   나. 의무부가 사유 : 상품특성인 Pay As You Use의 적용을 위해 스위치 On/Off

기능을 활용한 펫산책 횟수 산정

8. 의무가입에 관한 사항 (2)

 ...


In [3]:
# 3. 텍스트 분할 (Markdown)
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_core.documents import Document

# 마크다운 헤더 기준 분할
headers_to_split_on = [
    ("#", "Header1"),
    ("##", "Header2"),
    ("###", "Header3"),
    ("####", "Header4"),
]
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)

# 분할 적용
all_chunks = []
for doc in documents:
    md_chunks = md_splitter.split_text(doc.page_content)
    for chunk in md_chunks:
        all_chunks.append(Document(
            page_content=chunk.page_content,
            metadata={**doc.metadata, **chunk.metadata}
        ))

print(f"{len(all_chunks)}개 청크 생성")

# 샘플 확인
print(f"\n[메타데이터]: {all_chunks[0].metadata}")
print(f"[내용]:\n{all_chunks[0].page_content[:300]}...")

15021개 청크 생성

[메타데이터]: {'source': 'data\\petsurance_markdown\\Carrot\\Carrot_SmartON_PetWalkIns_Method.md'}
[내용]:
사업방법서  별지  
캐롯  스마트ON  펫산책보험  
1. 보험의 종류 : 특종/기타/동물  
2. 보험종목의 명칭 : 캐롯 스마트ON 펫산책보험  
3. 보험의 목적  
가. 법률상의 배상책임  
나.  각종 비용손해  
4. 보험기간 및 보험료 납입주기에 관한 사항  
가.  보험기간  :  1년  
나. 보험료 납입주기 : 일시납을 원칙으로 하되, 해외여행 스마트ON 특별약관을  
부가하는 계약의 경우 정산보험료를 영수할 수 있다.  
5. 배당에 관한 사항 : 배당금을 지급하지 않는다.  
6. 보험료 차등적용에 관한 사항...


In [4]:
# 4. 임베딩 모델 설정
from langchain_community.embeddings import HuggingFaceEmbeddings

# 한국어 특화 임베딩 모델
embeddings = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
    model_kwargs={"device": "cuda"}
)

print("임베딩 모델 로드 완료")

C:\Users\ksw\AppData\Local\Temp\ipykernel_65860\503297018.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


임베딩 모델 로드 완료


In [5]:
# 5. 벡터 스토어 저장
from langchain_community.vectorstores import Chroma

# 청크를 임베딩하여 벡터 스토어에 저장
vectorstore = Chroma.from_documents(
    documents=all_chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print(f"{len(all_chunks)}개 청크 벡터 스토어 저장 완료")

15021개 청크 벡터 스토어 저장 완료


In [6]:
# 6. 리트리버 설정
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 20}  
)

# 테스트 (키워드 필터링)
docs = retriever.invoke("슬관절 탈구 보장")
filtered = [d for d in docs if "슬관절" in d.page_content]

print(f"검색 결과: {len(docs)}개 중 '슬관절' 포함: {len(filtered)}개")
for i, doc in enumerate(filtered[:5]):
    print(f"\n--- [{i+1}] ---")
    print(doc.page_content[:300])

검색 결과: 20개 중 '슬관절' 포함: 16개

--- [1] ---
### 제47조  제1항  및  제3항의  정상인의  신체  각  관절에  대한  평균  운동가능영역
을  기준으로  정상각도  및  측정방법  등을  따른다.  
나) 관절기능장해를  표시할  경우  장해부위의  장해각도와  정상부위의  측정치를  동
시에  판단하여  장해상태를  명확히  한다.  단,  관절기능장해가  신경손상으로  인
한  경우에는  운동범위  측정이  아닌  근력  및  근전도  검사를  기준으로  평가한다.
7)  ‘관절  하나의  기능을  완전히  잃었을  때’라  함은  아래의  경우  중  하나에  

--- [2] ---
### 제47조  제1항  및  제3항의  정상인의  신체  각  관절에  대한  평균  운동가능영역
을  기준으로  정상각도  및  측정방법  등을  따른다.  
나) 관절기능장해를  표시할  경우  장해부위의  장해각도와  정상부위의  측정치를  동
시에  판단하여  장해상태를  명확히  한다.  단,  관절기능장해가  신경손상으로  인
한  경우에는  운동범위  측정이  아닌  근력  및  근전도  검사를  기준으로  평가한다.
7)  ‘관절  하나의  기능을  완전히  잃었을  때’라  함은  아래의  경우  중  하나에  

--- [3] ---
### 제47조  제1항  및  제3항의  정상인의  신체  각  관절에  대한  평균  운동가능영역
을  기준으로  정상각도  및  측정방법  등을  따른다.  
나) 관절기능장해를  표시할  경우  장해부위의  장해각도와  정상부위의  측정치를  동
시에  판단하여  장해상태를  명확히  한다.  단,  관절기능장해가  신경손상으로  인
한  경우에는  운동범위  측정이  아닌  근력  및  근전도  검사를  기준으로  평가한다.
7)  ‘관절  하나의  기능을  완전히  잃었을  때’라  함은  아래의  경우  중  하나에  

--- [4] ---
### 제47조  제1항  및  

In [7]:
# 7. 프롬프트 설정 
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 퓨샷 예시
examples = [
    {"question": "슬개골 탈구 수술비가 보장되나요?", "answer": "[상품명] 펫블리 반려견보험\n[근거] 제1조\n[조항] 제1조\n[답변] 네, 보장됩니다."},
    {"question": "보장하지 않는 손해는?", "answer": "[상품명] 펫블리 반려견보험\n[근거] 제5조\n[조항] 제5조\n[답변] 예방접종 등은 안 됩니다."}
]

# 예시 템플릿
example_prompt = ChatPromptTemplate.from_messages([("user", "{question}"), ("assistant", "{answer}")])

# 퓨샷 프롬프트
few_shot_prompt = FewShotChatMessagePromptTemplate(example_prompt=example_prompt, examples=examples)


# 최종 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 30년 경력의 펫보험 전문 상담사입니다.
제공된 [참고 문맥]과 [이전 대화 지식]을 바탕으로 답변하세요.
## 중요 지침
1. **'보험약관(_Terms.md)'** 문서를 최우선 순위로 참고하여 답변하세요.
2. '상품요약서(_Method.md)'나 '안내서'는 보조 자료로만 활용하세요.
3. 정보가 상충할 경우, 반드시 **'약관'**의 내용을 기준으로 답해야 합니다.
4. 확실하지 않은 정보는 "약관에서 확인되지 않습니다"라고 답하세요.
[참고 문맥]
{context}
[이전 대화 지식]
{kg_context}
## 답변 형식
[상품명] 해당 보험 상품명
[근거 및 조항] 관련 조항 내용 요약, 제N조 
[답변] 고객이 이해하기 쉬운 설명"""),
    few_shot_prompt,
    ("user", "{question}")
])

# 문서 포맷팅 함수
def format_docs(docs):
    formatted = []
    for d in docs:
        source = d.metadata.get('source', '알 수 없음')
        fname = source.split('\\')[-1] # 파일명만 추출
        formatted.append(f"[문서명: {fname}]\n{d.page_content}")
    return "\n\n---\n\n".join(formatted)

print("프롬프트 설정 완료")

프롬프트 설정 완료


In [8]:
# 8. LLM 모델 설정
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# Gemini 모델 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0  # 낮을수록 일관된 답변
)

# 출력 파서
output_parser = StrOutputParser()

print("LLM 모델 설정 완료")

LLM 모델 설정 완료


In [9]:
# 9. 대화 지식 그래프 메모리 설정
from langchain_community.memory.kg import ConversationKGMemory

# 대화에서 엔티티를 뽑아낼 때 llM 사용
memory = ConversationKGMemory(llm=llm, return_messages=True)

print("지식그래프 메모리 설정 완료")

지식그래프 메모리 설정 완료


In [10]:
# 10. RAG 체인 구성 및 실행 
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser


# 메모리에서 지식 정보(kg_context)를 읽어오는 함수
def get_kg_context(input_dict):
    # input_dict는 {"question": "..."} 형태입니다.
    return memory.load_memory_variables({"input": input_dict["question"]})["history"]

# RAG 체인 정의: 컨텍스트(Retriever) + 지식그래프(KG) + 질문 통합
rag_chain_with_kg = (
    {
        "context": (lambda x: x["question"]) | retriever | format_docs,        # 약관 문서 검색 및 포맷팅
        "question": RunnablePassthrough(),       # 사용자 질문 전달
        "kg_context": RunnableLambda(get_kg_context) # 지식그래프 메모리 데이터 로드
    }
    | prompt                                       # 프롬프트 템플릿 적용
    | llm                                          # Gemini 2.5 Flash 모델
    | StrOutputParser()                           # 출력 파싱
)

# 답변 생성 및 지식 자동 저장 함수
def ask(question):
    print(f"질문: {question}\n")
    full_answer = ""
    print("답변:", end=" ", flush=True)
    
    # 스트리밍 방식으로 답변 출력
    for chunk in rag_chain_with_kg.stream({"question": question}):
        print(chunk, end="", flush=True)
        full_answer += chunk
    
    # 대화 내용을 지식그래프 메모리에 저장 (엔티티 및 관계 추출 발생)
    memory.save_context({"input": question}, {"output": full_answer})
    print("\n" + "="*50)

# 테스트 실행
ask("DB손해보험 펫보험에서 강아지 슬관절 탈구 보장되나요?")

질문: DB손해보험 펫보험에서 강아지 슬관절 탈구 보장되나요?

답변: [상품명] 무배당 프로미라이프 다이렉트 펫블리 반려견보험2510(CM)
[근거 및 조항] 반려동물의료비Ⅴ(슬관절/고관절탈구포함(1년이후))(실손) 또는 반려동물의료비Ⅵ(슬관절/고관절탈구포함(1년이후))(실손) 특별약관의 보장 내용 (상품요약서 2. 보험금 지급사유 및 지급제한 사항 - 1) 보장(보상)의 종류, 보험금 지급사유 및 지급금액)
[답변] 네, DB손해보험의 펫블리 반려견보험에서는 강아지의 슬관절 탈구를 보장합니다. 다만, 슬관절 탈구 관련 질병 및 상해로 인한 치료비는 계약(부활(효력회복))일로부터 1년이 지난날의 다음날부터 보장됩니다.
